In [23]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import time

## WORDNET 이용한 유의어 추출하기

In [24]:
from nltk.corpus import wordnet as wn

search_keywords_list = ['you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife']

sim_words_list = []
for conf_word in search_keywords_list:
    for synset in wn.synsets(conf_word):
        sim_words_list.append(synset.name()) #동의, 유의어
        #sim_words_list.append(synset.hypernyms()) # 상위어
        #sim_words_list.append(synset.hyponyms()) # 하위어
        
conf_words_flttend_list = ''.join(str(sim_words_list)) #문자열로 변환
result = re.findall(r"(?i)\b[a-z]+\b",conf_words_flttend_list)
result_ =[]
for i in result:
    if i != 'n' and i != 'v' and i != 's' and i != 'a':
        result_.append(i)
        
search_keywords_list_re = set(result_)
search_keywords_list_re 

{'aunt',
 'baby',
 'beget',
 'brother',
 'buddy',
 'conserve',
 'counterpart',
 'cousin',
 'daughter',
 'don',
 'duplicate',
 'ex',
 'father',
 'forefather',
 'founder',
 'gemini',
 'grandchild',
 'granddaughter',
 'grandfather',
 'grandma',
 'he',
 'helium',
 'husband',
 'in',
 'law',
 'match',
 'mother',
 'nephew',
 'niece',
 'parent',
 'person',
 'rear',
 'sister',
 'son',
 'stepdaughter',
 'stepfather',
 'stepmother',
 'stepson',
 'twin',
 'uncle',
 'widow',
 'widower',
 'wife'}

In [25]:
def character_all_section(text):


    # Number of Characters
    def NumberofCharacters(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count



    # number_of_characters = NumberofCharacters(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # number_of_characters
    # print ('=============================================')
    # print ('Number of Characters :', number_of_characters)
    # print ('=============================================')

    ####################################
    #### Character Descriptiveness #####
    ####################################

    def character_descrip(text):

        input_sentence = text

        def findSentence(input_sentence):
            result = []

            data = str(input_sentence)
            #data = input_sentence.splitlines()
            
            findText = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']

            sentences = data.split(".")
            
            for sentence in sentences:
                for item in findText:
                    if item in sentence:
                        result.append(sentence)

            return result

        input_sent_included_character = findSentence(text) 
        input_sent_chr = set(input_sent_included_character) #중복값을 제거해보자
        input_sent_chr = '.'.join(input_sent_chr) #하나의 문자열로 합쳐야 원본 문장처럼 변환되고, 이것을 show/tell 분석코드에 넣게됨



        #입력된 전체 문장을 개별문장으로 분리하여 전처리 처리함
        def sentence_to_df(input_sentence):

            input_text_df = nltk.tokenize.sent_tokenize(input_sentence)
            test = []

            for i in range(0,len(input_text_df)):
                new_label = np.random.randint(0,2)  # 개별문장(input_text_df) 수만큼 0 또는 1 난수 생성
                data = [new_label, input_text_df[i]]
                test.append(data)

            #print(test)
            dataf = pd.DataFrame(test, columns=['label', 'text'])
            #print(dataf)
            return dataf


        class STDataset(Dataset):
            ''' Showing Telling Corpus Dataset '''
            def __init__(self, df):
                self.df = df

            def __len__(self):
                return len(self.df)

            def __getitem__(self, idx):
                text = self.df.iloc[idx, 1]
                label = int(self.df.iloc[idx, 0])
                return text, label


        ###########입력받은 데이터 처리 실행하는 메소드 showtell_classfy() ###############
        #result_all.html에서 입력받을 text를 contents에 넣고 전처리 후 데이터프레임에 넣어줌
        def showtell_classfy(text):
            contents = str(text)
            preprossed_contents_df = sentence_to_df(contents)

            preprossed_contents_df.dropna(inplace=True)
            #전처리된 데이터를 확인(데이터프레임으로 가공됨)
            preprossed_contents_df__ = preprossed_contents_df.sample(frac=1, random_state=999)
            

            #파이토치에 입력하기 위해서 로딩...
            ST_test_dataset = STDataset(preprossed_contents_df__)
            test_loader = DataLoader(ST_test_dataset, batch_size=1, shuffle=True, num_workers=0)
            #로딩되는지 확인
            ST_test_dataset.__getitem__(1)

            #time.sleep(1)



            #check whether cuda is available
            #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
            device = torch.device("cpu")  
            #device = torch.device("cuda")
            #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
            tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
            model = BertForSequenceClassification.from_pretrained('bert-large-cased')
            model.to(device)



            # for text, label in test_loader :
            #     print("text:",text)
            #     print("label:",label)


            #저장된 모델을 불러온다.
            #J:\Django\EssayFit_Django\essayfitaiproject\essayfitapp\model.pt
            #time.sleep(1)
            #model = torch.load("/Users/jongphilkim/Desktop/Django_WEB/essayfitaiproject/essayai/model.pt", map_location=torch.device('cpu'))
            model = torch.load("model.pt", map_location=torch.device('cpu'))
            print("model loadling~")
            model.eval()


            pred_loader = test_loader
            print("pred_loader:", pred_loader)
            total_loss = 0
            total_len = 0
            total_showing__ = 0
            total_telling__ = 0

            showing_conunter = [] #문장에 해당하는 SHOWING을 계산한다.
            
            print("check!")
            for text, label in pred_loader:
                print("text:",text)
                #print("label:",label)
                encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] #text to tokenize
                padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] #padding
                sample = torch.tensor(padded_list) #torch tensor로 변환
                sample, label = sample.to(device), label.to(device) #tokenized text에 label을 넣어서 Device(gpu/cpu)에 넣기 위해 준비
                labels = torch.tensor(label) #레이블을 텐서로 변환
                #time.sleep(1)
                outputs = model(sample,labels=labels) #모델을 통해서 샘플텍스트와 레이블 입력데이터를 출력 output에 넣음
                #시간 딜레이를 주자
                #time.sleep(1)
                _, logits = outputs #outputs를 로짓에 넣음 이것을 softmax에 넣으면 0~1 사이로 결과가 출력됨
                
                pred = torch.argmax(F.softmax(logits), dim=1) #드디어 예측한다. argmax는 리스트(계산된 값)에서 가장 큰 값을 추출하여 pred에 넣는다. 0 ~1 사이의 값이 나올거임
                print('pred :', pred)
                # correct = pred.eq(labels) 
                showing__ = pred.eq(1) # 예측한 결과가 1과 같으면 showing이다   >> TRUE   SHOWING을 추출하려면 이것만 카운드하면 된다. 
                telling__ = pred.eq(0) # 예측한 결과가 0과 같으면 telling이다   >> FALSE
                
                #print('showing : ', showing__)
                #print('telling : ', telling__)
                
                
                showing_conunter.append(text)        
                #pred_ = round(float(pred))
                showing_conunter.append(pred)



            return showing_conunter 


        st_re = showtell_classfy(str(input_sent_chr)) # 캐릭터거 포함된 문장(전처리 완료된) 입력

        df = DataFrame(st_re)
        df_ = df[0::2] # 글만 추출
        df_label = df[1::2] # 레이블만 추출

        df_.reset_index(drop=True, inplace=True) #데이터를 합치기 위해서 초기화
        df_label.reset_index(drop=True, inplace=True)

        df_result = pd.concat([df_,df_label],axis=1) #합치기

        df_result.columns = ['sentence','show/tell']

        df_fin = df_result['show/tell'].value_counts(normalize=True)
        list(df_fin)
        showing_sentence_with_char = max(round(df_fin*100))

        # print("===============================================================")
        # print ('Character Descriptiveness : ', showing_sentence_with_char)
        # print("===============================================================")

        return showing_sentence_with_char



    ################################################
    #############  Emphasis on YOU  ################
    ################################################
    def EmphasisOnYou(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'I', 'my', 'me', 'mine', 'one']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count


    # EmphasisOnYou_ = EmphasisOnYou(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnYou_
    # print ('=============================================')
    # print ('Emphasis on You :', EmphasisOnYou_)
    # print ('=============================================')



    #########################################
    ######### Emphasis on others  ###########
    #########################################
    def EmphasisOnOthers(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin',
                        'daughter', 'duplicate', 'ex', 'forefather', 'founder', 'gemini',
                        'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium', 'husband',
                        'in', 'law', 'match', 'mother', 'nephew', 'niece', 'parent', 'person', 'rear',
                        'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow',
                        'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
    
    
        return char_total_count


    # EmphasisOnOthers_ = EmphasisOnOthers(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnOthers_
    # print ('=============================================')
    # print ('Emphasis on Others :', EmphasisOnOthers_)
    # print ('=============================================')


    character_descriptiveness = character_descrip(text)
    print("===============================================================")
    print ('Character Descriptiveness : ' , character_descriptiveness)
    print("===============================================================")


    number_of_characters = NumberofCharacters(text) 
    print ('=============================================')
    print ('Number of Characters :' , number_of_characters)
    print ('=============================================')


    EmphasisOnYou_ = EmphasisOnYou(text)
    print ('=============================================')
    print ('Emphasis on You :' , EmphasisOnYou_)
    print ('=============================================')


    EmphasisOnOthers_ = EmphasisOnOthers(text) 
    print ('=============================================')
    print ('Emphasis on Others :' , EmphasisOnOthers_)
    print ('=============================================')


    return character_descriptiveness, number_of_characters, EmphasisOnYou_, EmphasisOnOthers_



## 1000명의 개별 에세이 값을 각각 불러와서, 데이터프레임으로 결과값을 저장하고, 표준편차 계산하여 적용

In [26]:
#!/usr/bin/python
# -*- coding:utf-8 -*-

import json
import pandas as pd
import sys
import os

In [27]:
pwd

'C:\\Users\\cacki\\Projects\\008_AI\\Story_Analysis-master8\\Story_Analysis-master\\CHARACTER_'

In [28]:
path = "C:\\Users\\cacki\\Projects\\008_AI\\Story_Analysis-master8\\Story_Analysis-master\\CHARACTER_\\ps_data\\"
file_list = os.listdir(path)
file_list

['each_personal_essay_0.json',
 'each_personal_essay_1.json',
 'each_personal_essay_10.json',
 'each_personal_essay_100.json',
 'each_personal_essay_101.json',
 'each_personal_essay_102.json',
 'each_personal_essay_103.json',
 'each_personal_essay_104.json',
 'each_personal_essay_105.json',
 'each_personal_essay_106.json',
 'each_personal_essay_107.json',
 'each_personal_essay_108.json',
 'each_personal_essay_109.json',
 'each_personal_essay_11.json',
 'each_personal_essay_110.json',
 'each_personal_essay_116.json',
 'each_personal_essay_117.json',
 'each_personal_essay_118.json',
 'each_personal_essay_119.json',
 'each_personal_essay_12.json',
 'each_personal_essay_120.json',
 'each_personal_essay_121.json',
 'each_personal_essay_122.json',
 'each_personal_essay_123.json',
 'each_personal_essay_124.json',
 'each_personal_essay_125.json',
 'each_personal_essay_126.json',
 'each_personal_essay_127.json',
 'each_personal_essay_128.json',
 'each_personal_essay_129.json',
 'each_personal_e

In [29]:
from tqdm import tqdm

In [30]:
essay_results = []

for file_name_raw in tqdm(file_list): # 파일이름을 하나씩 불러와서
    file_name = path  + file_name_raw #경로를 추가하고
    with open(file_name, 'r') as json_file : #JSON파일로 파일을 열어서
        ps_input_each_data = json.load(json_file) # JSON파일로 로드한다.
        input_data = json.dumps(ps_input_each_data)
        re_each_ess = character_all_section(input_data)
        essay_results.append(re_each_ess)

  0%|          | 0/971 [00:00<?, ?it/s]

model loadling~
pred_loader: <torch.utils.data.dataloader.DataLoader object at 0x00000246AF7F16C8>
check!
text: ('I have had to practically wrestle the pride out of my parents to get them to communicate on every little aspect of my life.',)


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:210: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pred : tensor([1])
text: ('"}, {"index": 1, "personal_essay": "My ideal college is a combination of challenging and supportive.',)
pred : tensor([1])
text: ('Myschool has become my safe place, the window where I can see my reflection, despite the blurring trees in the background.',)
pred : tensor([1])
text: ('I am not sure how my future will include all of them, but I know it will, and I know I will also find new interests along the way',)
pred : tensor([1])
text: ('Two of my favorite art galleries I visit regularly are the Yale art gallery and the New Museum.My favorite movie I saw recently is Someone Great.',)
pred : tensor([1])
text: ('The article also mentioned College Horizons, which I would be attending in 2 months at the time.',)
pred : tensor([1])
text: ('My mom is Danish and her entire family still lives in Denmark.',)
pred : tensor([0])
text: ('When I am in a rut I will look at my saved photos and copy or riff off of them.',)
pred : tensor([1])
text: ('Most of the material I 

pred : tensor([1])
text: ('My interests all feel very different from each other, but all three areas are important to me.',)
pred : tensor([1])
text: ('(150 words or less)*List the titles of the books you read for pleasure that you enjoyed most in the past year.',)
pred : tensor([1])
text: ('{"schema": {"fields": [{"name": "index", "type": "integer"}, {"name": "personal_essay", "type": "string"}], "primaryKey": ["index"], "pandas_version": "0.',)
pred : tensor([1])
text: ('I use many different social media platforms to connect with my friends and for inspiration.',)
pred : tensor([1])
text: ('Being Native American, I am looking for a school that will have a strong cultural center that I can be a part of.',)
pred : tensor([1])
text: ('I have realized that the beauty of my circumstance is that I must work so hard to keep my halves together and the perseverance and initiative I have learned are invaluable.',)
pred : tensor([1])
text: ('With my own personal ties to Native communities, I fe

...

pred : tensor([0])
text: ('I am highly motivated to inspire others to do their best.',)
pred : tensor([0])
text: (')I am guided, not just by a motivation to tackle the problems of inequity and injustice, but to do so in a cooperative manner that results in enduring and effective solutions.',)
pred : tensor([1])
text: ("When you come upon a mountain, you only need resolve and commitment toclimb it.When I was looking at high schools, I chose Strake Jesuit, not just because of the great education it offered, but also because of the motto 'Men for Others'.",)
pred : tensor([1])
text: ('As we stood with her in the cold hospital room, memories of the timeswhen she would pick me up from school on Thursdays with my grandfather cameflooding back.',)
pred : tensor([1])
text: ('Even when one can afford care, like my family, the system does not prioritize easy access.',)
pred : tensor([1])
text: ('She died later that night.',)
pred : tensor([0])
text: ('I want the place I live to be filled with pe

pred : tensor([1])
text: ("His wife had been focused on meeting his health care needs and wasn't thinking about their house.",)
pred : tensor([0])
text: ('I put all my energy into recovery, and my determination and hard work paid off.',)
pred : tensor([1])
text: ('This has happened for centuries\\u2014remember the cholera outbreaks in England that were caused by poor public water systems.I spent the majority of my time after school, every weekend during the school year, and several weeks each summer debating.',)
pred : tensor([1])
text: ("I thought to myself, 'This has happened to me before.I am interested in a biology major and appreciate the various concentrations that are offered within that major.",)
pred : tensor([1])
text: ('I waded into homes filled withchest-deep water and rescued 20 people, including several small children and their pets.',)
pred : tensor([1])
text: ('In many ways, I think this may be the first step in global cooperation in setting and maintaining global stand

C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  4%|▍         | 40/971 [2:31:46<58:52:42, 227.67s/it]

['parents', 'grandmother', 'in', 'it', 'person', 'husband', 'they', 'match', 'grandfather', 'your', 'their', 'i', 'he', 'father', 'myself', 'him', 'one', 'me', 'mother', 'wife', 'my', 'them', 'her', 'you']


KeyError: "word 'father' not in vocabulary"

In [31]:
essay_results

[(88.0, 387, 185, 100),
 (85.0, 258, 122, 73),
 (91.0, 282, 125, 85),
 (84.0, 234, 109, 69),
 (49.0, 600, 247, 210),
 (79.0, 438, 172, 150),
 (94.0, 288, 158, 54),
 (40.0, 671, 359, 195),
 (89.0, 68, 25, 27),
 (88.0, 303, 128, 97),
 (87.0, 647, 259, 255),
 (84.0, 561, 265, 148),
 (19.0, 2117, 866, 710),
 (88.0, 468, 198, 177),
 (68.0, 773, 322, 261),
 (81.0, 284, 156, 50),
 (89.0, 644, 297, 209),
 (86.0, 329, 164, 97),
 (87.0, 286, 139, 80),
 (87.0, 211, 114, 32),
 (82.0, 255, 103, 89),
 (97.0, 448, 206, 149),
 (87.0, 467, 206, 162),
 (23.0, 381, 176, 102),
 (59.0, 718, 310, 252),
 (83.0, 233, 97, 86),
 (97.0, 512, 212, 166),
 (86.0, 161, 87, 28),
 (59.0, 599, 234, 250),
 (85.0, 207, 102, 59),
 (100.0, 0, 0, 0),
 (92.0, 372, 168, 114),
 (41.0, 1426, 656, 433),
 (95.0, 602, 267, 174),
 (83.0, 623, 285, 204),
 (49.0, 805, 360, 215),
 (92.0, 168, 69, 67),
 (84.0, 361, 177, 94),
 (70.0, 579, 264, 198),
 (92.0, 366, 184, 104)]

In [32]:
# ===============================================================
# Character Descriptiveness :  33.0
# ===============================================================
# ['their', 'they', 'me', 'you', 'her', 'it', 'myself', 'i', 'my', 'them']
# ai_character_section.py:92: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Number of Characters : 92
# =============================================
# ['i', 'my', 'me']
# ai_character_section.py:338: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Emphasis on You : 60
# =============================================
# ['their', 'they', 'you', 'her', 'it', 'myself', 'them']
# ai_character_section.py:409: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Emphasis on Others : 32
# =============================================
# (33.0, 92, 60, 32)

In [33]:
df_r = pd.DataFrame(essay_results, columns =['char_desc', 'num_chars', 'emph_you', 'emph_others'])  #e데이터프레임에 넣기
df_r

,char_desc,num_chars,emph_you,emph_others
0,88.0,387,185,100
1,85.0,258,122,73
2,91.0,282,125,85
3,84.0,234,109,69
4,49.0,600,247,210
5,79.0,438,172,150
6,94.0,288,158,54
7,40.0,671,359,195
8,89.0,68,25,27
9,88.0,303,128,97


In [34]:
#1000명 데이터의 각 값의 평균 계산
char_desc_mean = df_r['char_desc'].mean()
char_desc_mean

77.975

In [35]:
num_chars_mean = df_r['num_chars'].mean()
num_chars_mean

478.3

In [36]:
emph_you_mean = df_r['emph_you'].mean()
emph_you_mean 

214.325

In [37]:
emph_others_mean = df_r['emph_others'].mean()
emph_others_mean

150.625

In [38]:
# 30명의 최종 평균값 계산
char_re_30 = [char_desc_mean, num_chars_mean, emph_you_mean, emph_others_mean]
char_re_30

[77.975, 478.3, 214.325, 150.625]

## 1명의 입력데이터

In [ ]:
input_text = """Bloomington Normal is almost laughably cliché for a midwestern city. Vast swathes of corn envelop winding roads and the heady smell of BBQ smoke pervades the countryside every summer. Yet, underlying the trite norms of Normal is the prescriptive force of tradition—the expectation to fulfill my role as a female Filipino by playing Debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.So when I discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, I was fascinated by the music's novelty. Jazz guitar was not only evocative and creative, but also strangely liberating. I began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. And, although I did not know what would be the 'best' route for me to follow as a musician, the freedom to forge whatever path I felt was right seemed to be exactly what I needed; there were no expectations for me to continue in any particular way—only the way that suited my own desires.While journeying this trail, I found myself at Interlochen Arts Camp the summer before my junior year. Never before had I been immersed in an environment so conducive to musical growth: I was surrounded by people intensely passionate about pursuing all kinds of art with no regard for ideas of what art 'should' be. I knew immediately that this would be a perfect opportunity to cultivate my sound, unbounded by the limits of confining tradition. On the first day of camp, I found that my peer guitarist in big band was another Filipino girl from Illinois. Until that moment, my endeavors in jazz guitar had been a solitary effort; I had no one with whom to collaborate and no one against whom I could compare myself, much less someone from a background mirroring my own. I was eager to play with her, but while I quickly recognized a slew of differences between us—different heights, guitars, and even playing styles—others seemed to have trouble making that distinction during performances. Some even went as far as calling me 'other-Francesca.' Thus, amidst the glittering lakes and musky pine needles of Interlochen, I once again confronted Bloomington's frustrating expectations.After being mistaken for her several times, I could not help but view Francesca as a standard of what the 'female Filipino jazz guitarist' should embody. Her improvisatory language, comping style and even personal qualities loomed above me as something I had to live up to. Nevertheless, as Francesca and I continued to play together, it was not long before we connected through our creative pursuit. In time, I learned to draw inspiration from her instead of feeling pressured to follow whatever precedent I thought she set. I found that I grew because of, rather than in spite of, her presence; I could find solace in our similarities and even a sense of comfort in an unfamiliar environment without being trapped by expectation. Though the pressure to conform was still present—and will likely remain present in my life no matter what genre I'm playing or what pursuits I engage in—I learned to eschew its corrosive influence and enjoy the rewards that it brings. While my encounter with Francesca at first sparked a feeling of pressure to conform in a setting where I never thought I would feel its presence, it also carried the warmth of finding someone with whom I could connect. Like the admittedly trite conditions of my hometown, the resemblances between us provided comfort to me through their familiarity. I ultimately found that I can embrace this warmth while still rejecting the pressure to succumb to expectations, and that, in the careful balance between these elements, I can grow in a way that feels both like discove"""

In [ ]:
ps_char_re = character_all_section(input_text)
ps_char_re 

In [ ]:
type(ps_char_re )

In [ ]:
one_ps_char_desc = ps_char_re[0] #튜플에서 첫번재 인댁스 값 가져오기 : Character Descriptiveness

# 결과도출 1명의 Character Descriptiveness 계산결과
one_ps_char_desc

In [ ]:
ideal_mean = char_desc_mean #1000명 데이터의 각 값의 평균 값 전달

In [ ]:
min_ = int(ideal_mean-ideal_mean*0.6)
print('min_', min_)
max_ = int(ideal_mean+ideal_mean*0.6)
print('max_: ', max_)
div_ = int(((ideal_mean+ideal_mean*0.6)-(ideal_mean-ideal_mean*0.6))/3)
print('div_:', div_)


In [ ]:
cal_abs = abs(ideal_mean - one_ps_char_desc) # 개인 - 단체 값의 절대값계산

print('cal_abs 절대값 :', cal_abs)
compare = (one_ps_char_desc + ideal_mean)/7
print('compare :', compare)

if one_ps_char_desc > ideal_mean: # 개인점수가 평균보다 클 경우는 overboard
    if cal_abs > compare: # 개인점수가 개인평균차의 절대값보다 클 경우, 즉 차이가 많이 날경우
        print("Overboard")
    else: #차이가 많이 안나면
        print("Ideal")
        
    
elif one_ps_char_desc < ideal_mean: # 개인점수가 평균보다 작을 경우 lacking
    if cal_abs > compare: #차이가 많이나면 # 개인점수가  평균보다 작을 경우 Lacking이고 
        print("Lacking")
    else: #차이가 많이 안나면
        print ("Ideal")
        
else:
    print("Ideal")

    
    

## 1000명 에세이 데이터 처리 결과 저장

In [ ]:
import pickle
# save
with open('1000_essay_results.pickle', 'wb') as f:
    pickle.dump(essay_results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load
with open('data_ai_theme.pickle', 'rb') as f:
    result_most_simWords = pickle.load(f)